In [10]:
import os
import numpy as np
import mne
%matplotlib qt
import matplotlib
import matplotlib.pyplot as plt

In [11]:
# Load the data
from mne_bids import (BIDSPath,read_raw_bids)

# path where to save the datasets.
bids_root = "Dataset/ds004147-filtered"
subject_id = '27'


bids_path = BIDSPath(subject=subject_id,task="casinos",
                     datatype='eeg', suffix='eeg',
                     root=bids_root)

# read the file
raw = read_raw_bids(bids_path)
# fix the annotations readin
# ccs_eeg_utils.read_annotations_core(bids_path,raw)

# raw.load_data()
;

Extracting parameters from Dataset\ds004147-filtered\sub-27\eeg\sub-27_task-casinos_eeg.vhdr...
Setting channel info structure...
Reading events from Dataset\ds004147-filtered\sub-27\eeg\sub-27_task-casinos_events.tsv.
The event "Stimulus" refers to multiple event values. Creating hierarchical event names.
    Renaming event: Stimulus -> Stimulus/S  1
    Renaming event: Stimulus -> Stimulus/S  2
    Renaming event: Stimulus -> Stimulus/S  3
    Renaming event: Stimulus -> Stimulus/S  4
    Renaming event: Stimulus -> Stimulus/S  7
    Renaming event: Stimulus -> Stimulus/S  1
    Renaming event: Stimulus -> Stimulus/S  2
    Renaming event: Stimulus -> Stimulus/S  3
    Renaming event: Stimulus -> Stimulus/S  5
    Renaming event: Stimulus -> Stimulus/S  7
    Renaming event: Stimulus -> Stimulus/S  1
    Renaming event: Stimulus -> Stimulus/S  2
    Renaming event: Stimulus -> Stimulus/S  3
    Renaming event: Stimulus -> Stimulus/S  4
    Renaming event: Stimulus -> Stimulus/S  7
  

''

In [ ]:
# annotating points of >40uV difference between samples
raw2 = raw.copy()
raw2.load_data()
raw2.filter(l_freq=0.1,h_freq=50)
# for each channel
# raw[chan] set bad annotation at indices = np.where(abs(np.diff(raw[chan][0][0]))>40e-6)[0]
all_onsets = []
all_durations = []
all_descriptions = []
new_anno = raw2.annotations.copy()
for chan in raw2.ch_names:
    bad_indices = np.where(abs(np.diff(raw[chan][0][0]))>40e-6)[0]
    n_anno = len(bad_indices)
    times = raw[chan][1][bad_indices]
    new_anno = new_anno + mne.Annotations(
        onset=times,  # in seconds
        duration=[0.001]*n_anno,  # in seconds, too
        description=["BAD_" + chan + ": 40uV between samples"]*n_anno,
        orig_time=raw2.annotations.orig_time
    )
print(new_anno)
raw2.set_annotations(new_anno)
raw2.annotations
# mne.viz.plot_raw(raw2, duration=15, lowpass=50, highpass=1, scalings={"eeg":100e-6},n_channels=len(raw.ch_names))

In [ ]:
# plot for inspection - simple raw data
# mne.viz.plot_raw(raw, duration=150,highpass=1,lowpass=50, scalings={"eeg":100e-6},n_channels=len(raw.ch_names))

# figure a paper

In [57]:
# evoked - loading and grand averages + plotting

derivs_path = bids_root+"/derivatives/mne-bids-pipeline/"#"Dataset/ds004147-filtered-update/"

evoked_ave_all = []
cond_list = [ # contrasts between win and loss for each task-cue combination
    'Win LL',
    'Loss LL',
    'Win ML',
    'Loss ML',
    'Win MH',
    'Loss MH',
    'Win HH',
    'Loss HH'
]

sub_list = ["27","28","29","30","31","32","33","34","35","36","37","38"]
# sub_list = ["35"]
grand_avgs = []
# for each condition (task-cue combination), create a list of evoked responses across all subjects and then average
for i in range(0,len(cond_list)):
    print(i)
    for sub in sub_list:
        evoked_ave_all.append([])
        evoked_ave_all[i].append(mne.Evoked(derivs_path+"sub-"+sub+"/eeg/sub-"+sub+"_task-casinos_ave.fif",proj=False,verbose=False,condition=cond_list[i]))
    grand_avgs.append(mne.grand_average(evoked_ave_all[i])) # for this condition, average across subjects
    grand_avgs[i].comment=cond_list[i] # name of the condition/difference curve, so that it shows up in the plot

print(len(sub_list), len(evoked_ave_all), len(grand_avgs)) # just to check if the right number of results come out

mne.viz.plot_compare_evokeds(
    grand_avgs,
    combine="mean", # average erp across channels
    picks="FCz", # if we want to see only FCz
    colors=
         {'Win LL':'r',
         'Win ML':'r',
         'Win MH':'r',
         'Win HH':'r',
         'Loss LL':'b',
         'Loss ML':'b',
         'Loss MH':'b',
         'Loss HH':'b'},
    linestyles=
         {'Win LL':'solid',
         'Win ML':'dashed',
         'Win MH':'dashdot',
         'Win HH':'dotted',
         'Loss LL':'solid',
         'Loss ML':'dashed',
         'Loss MH':'dashdot',
         'Loss HH':'dotted'},
         ylim=dict(eeg=[-5, 20]),
         title=('sub: ',sub_list),
    time_unit="ms",
)

0
Identifying common channels ...
1
Identifying common channels ...
2
Identifying common channels ...
3
Identifying common channels ...
4
Identifying common channels ...
5
Identifying common channels ...
6
Identifying common channels ...
7
Identifying common channels ...
12 96 8
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"


C:\Users\Max\AppData\Local\Temp\ipykernel_26116\3966446996.py:31: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  mne.viz.plot_compare_evokeds(


[<Figure size 800x600 with 2 Axes>]

# rewp purple plot

In [59]:
# evoked - loading and grand averages + plotting

derivs_path = bids_root+"/derivatives/mne-bids-pipeline/"#"Dataset/ds004147-filtered-update/"

evoked_ave_all = []
cond_list = [ # contrasts between win and loss for each task-cue combination
    'Win LL - Loss LL',
    'Win ML - Loss ML',
    'Win MH - Loss MH',
    'Win HH - Loss HH'
]

sub_list = ["27","28","29","30","31","32","33","34","35","36","37","38"]
grand_avgs = []
# for each condition (task-cue combination), create a list of evoked responses across all subjects and then average
for i in range(0,len(cond_list)):
    print(i)
    for sub in sub_list:
        evoked_ave_all.append([])
        evoked_ave_all[i].append(mne.Evoked(derivs_path+"sub-"+sub+"/eeg/sub-"+sub+"_task-casinos_ave.fif",proj=False,verbose=False,condition=cond_list[i]))
    grand_avgs.append(mne.grand_average(evoked_ave_all[i])) # for this condition, average across subjects
    grand_avgs[i].comment=cond_list[i] # name of the condition/difference curve, so that it shows up in the plot

print(len(sub_list), len(evoked_ave_all), len(grand_avgs)) # just to check if the right number of results come out

print('cond_list', cond_list)

mne.viz.plot_compare_evokeds(
    grand_avgs,
    combine="mean", # average erp across channels
    picks="FCz", # if we want to see only FCz
    colors=['#EE82EE','#EE82EE','#a020f0','#a020f0'],
    linestyles=['solid','dashed','dotted','dashdot'],
    ylim=dict(eeg=[-5, 10]),
    title=('sub: ',sub_list),
    time_unit="ms",
)

0
Identifying common channels ...
1
Identifying common channels ...
2
Identifying common channels ...
3
Identifying common channels ...
12 48 4
cond_list ['Win LL - Loss LL', 'Win ML - Loss ML', 'Win MH - Loss MH', 'Win HH - Loss HH']
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"


C:\Users\Max\AppData\Local\Temp\ipykernel_26116\1594297936.py:28: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  mne.viz.plot_compare_evokeds(


[<Figure size 800x600 with 2 Axes>]